<a target="_blank" href="https://colab.research.google.com/github/gox6/colab-demos/blob/main/data-science-stack/polars-a-modern-dataframe-library.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<html>
<body>
    <div style="display: flex;">
        <a href="https://medium.com/@jgrygolec/polars-a-modern-dataframe-library-dcce43f43f3b" style="width: 45em; height: 15em; background-color: white; padding-left: 1em; display: flex; flex-direction: column; justify-content: center; color: inherit; text-decoration: none;">
            <p style="margin: 0; font-size: 1.5em; color: darkblue; font-weight: bold;">The notebook contains the code for Medium article</p>
            <p style="margin: 0; font-weight: bold; font-size: 1.5em; color: #3D4849;">Polars: A Modern DataFrame Library</p>
        </a>
        <a href="https://medium.com/@jgrygolec/polars-a-modern-dataframe-library-dcce43f43f3b" style="width: 15em; height: 15em;">
            <img src="https://cdn-images-1.medium.com/max/800/1*Oe1Jsf_23vUOma_2v6z75Q.png" style="width: 100%; height: 100%; object-fit: cover;">
        </a>
    </div>
</body>
</html>

# Project Setup

In [24]:
# Installing Python packages quietly

!pip install --quiet \
  datasets \
  duckdb \
  httpx \
  pandas \
  plotly \
  polars \
  pyarrow \
  pympler \
  1> /dev/null

!pip install nbformat 1> /dev/null


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [25]:
# Importing the packages
from io import BytesIO
import os
from pathlib import Path
import time
from zipfile import ZipFile

from datasets import Dataset
import duckdb
import httpx
import multiprocessing
import pandas as pd
import plotly.express as px
import polars as pl
import polars.selectors as cs
from pympler import asizeof

In [3]:
# Managing secrets
# - If using Colab please use Colab Secrets
# - If running outside Colab please provide secrets as environmental variables
COLAB = os.getenv("COLAB_RELEASE_TAG") is not None

if COLAB:
    from google.colab import userdata, data_table

    runtime_info = "Colab runtime"

    # Enabling Colab's data formatter for pandas
    data_table.enable_dataframe_formatter()
else:
    runtime_info = "Non Colab runtime"

print(runtime_info)

Non Colab runtime


# Interoperability

In [28]:
# Creating a Polars DataFrame
example = pl.DataFrame(
    {"foo": [1, 2, 3], 
    "pi": [3.14, 3.14, 3.14],
    "greeting": ["Hi", "Hello", "Bye"]}
    )

# From Polars DataFrames to other data structures
example_pd = example.to_pandas() # to Pandas
example_pa = example.to_arrow() # to PyArrow
example_np = example.to_numpy() # to NumPy
example_hf = Dataset(example.to_arrow()) # to Hugging Face
example_duckdb = duckdb.sql("select * from example") # to DuckDB

# From other data structures to Polars DataFrames
example_from_pd = pl.from_pandas(example_pd) # from Pandas
example_from_pa = pl.from_arrow(example_pa) # from PyArrow
example_from_np = pl.from_numpy(example_np) # from NumPy
example_from_hf = pl.from_arrow(example_hf.data.table) # from Hugging Face
example_from_duckdb =example_duckdb.pl() # from DuckDB

shape: (3, 3)
┌─────┬──────┬──────────┐
│ foo ┆ pi   ┆ greeting │
│ --- ┆ ---  ┆ ---      │
│ i64 ┆ f64  ┆ str      │
╞═════╪══════╪══════════╡
│ 1   ┆ 3.14 ┆ Hi       │
│ 2   ┆ 3.14 ┆ Hello    │
│ 3   ┆ 3.14 ┆ Bye      │
└─────┴──────┴──────────┘


In [34]:
duckdb.sql("""SELECT 
                  t1.*, 
                  max(t1.foo) over() as foo_max 
              FROM example t1
           """)

┌───────┬────────┬──────────┬─────────┐
│  foo  │   pi   │ greeting │ foo_max │
│ int64 │ double │ varchar  │  int64  │
├───────┼────────┼──────────┼─────────┤
│     1 │   3.14 │ Hi       │       3 │
│     2 │   3.14 │ Hello    │       3 │
│     3 │   3.14 │ Bye      │       3 │
└───────┴────────┴──────────┴─────────┘

In [33]:
res = pl.SQLContext(example=example).execute(
    ("""SELECT 
                  t1.*, 
                  max(t1.foo) over() as foo_max 
              FROM example t1
           """)
)
res.collect()

ComputeError: at least one key is required in a group_by operation

In [4]:
# Download a zip file into the buffer

if not Path("./data/MetroPT3(AirCompressor).csv").exists():
    url = "https://archive.ics.uci.edu/static/public/791/metropt+3+dataset.zip"
    with httpx.Client() as client:
        response = client.get(url)
        buffer = BytesIO(response.content)

    # Unzip into the current directory
    with ZipFile(buffer) as zip_ref:
        # Extract all files to the current directory
        zip_ref.extractall("./data/")

In [8]:
# We need to load the data
df_pl = pl.read_csv("./data/MetroPT3(AirCompressor).csv", try_parse_dates=True)

# It is faster to convert to pandas from polars, than load with pandas
df_pd = df_pl.to_pandas()

In [10]:
df_pd.head(5)

,,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,0,2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
1,10,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2,20,2020-02-01 00:00:19,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
3,30,2020-02-01 00:00:29,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
4,40,2020-02-01 00:00:39,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [7]:
df_pl.head(5)

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""2020-02-01 00:00:00""",-0.012,9.358,9.34,-0.024,9.358,53.6,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
10,"""2020-02-01 00:00:10""",-0.014,9.348,9.332,-0.022,9.348,53.675,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
20,"""2020-02-01 00:00:19""",-0.012,9.338,9.322,-0.022,9.338,53.6,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
30,"""2020-02-01 00:00:29""",-0.012,9.328,9.312,-0.022,9.328,53.425,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
40,"""2020-02-01 00:00:39""",-0.012,9.318,9.302,-0.022,9.318,53.475,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [6]:
df_pl.shape

(1516948, 17)

In [6]:
df_pl.describe()

statistic,,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1.516948e6,"""1516948""",1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6,1.516948e6
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",7.584735e6,null,1.367826,8.984611,7.568155,0.055956,8.985233,62.644182,2.050171,0.836957,0.160611,0.919848,0.832664,0.00342,0.991437,0.904156,0.937107
"""std""",4.3791e6,null,3.25093,0.639095,3.3332,0.382402,0.638307,6.516261,2.302053,0.369405,0.367172,0.271528,0.373276,0.058381,0.092141,0.294378,0.242771
"""min""",0.0,"""2020-02-01 00:00:00""",-0.032,0.73,-0.036,-0.032,0.712,15.4,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",3.79237e6,null,-0.014,8.492,8.254,-0.022,8.494,57.775,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
"""50%""",7.58474e6,null,-0.012,8.96,8.784,-0.02,8.96,62.7,0.045,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
"""75%""",1.13771e7,null,-0.01,9.492,9.374,-0.018,9.492,67.25,3.8075,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
"""max""",1.516947e7,"""2020-09-01 03:59:50""",10.676,10.302,10.288,9.844,10.3,89.05,9.295,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# CPU Benchmarking

In [7]:
# Function to measure the time it takes to run a function, and store the results


def measure_time(func, trials, metadata, container, *args, **kwargs):
    # Measure the time it takes to run the function
    times = []
    for _ in range(trials):
        start_time = time.perf_counter()
        func(*args, **kwargs)
        end_time = time.perf_counter()
        times.append(end_time - start_time)

    interim = pl.DataFrame(
        {"time": times, "package": metadata["package"], "task": metadata["task"]}
    )
    container = pl.concat([container, interim])

    return container

In [8]:
# Defining prerequisites for the benchmarking
vars_of_interest = [
    "TP2",
    "TP3",
    "H1",
    "DV_pressure",
    "Reservoirs",
    "Oil_temperature",
    "Motor_current",
]
vars_of_interest_pd = {var: "mean" for var in vars_of_interest}

In [9]:
# Defining test functions

def read_csv_pl():
    return pl.read_csv("./data/MetroPT3(AirCompressor).csv")


def read_csv_pd():
    return pd.read_csv("./data/MetroPT3(AirCompressor).csv")


def describe_pl():
    return df_pl.describe()


def describe_pd():
    return df_pd.describe()


def filter_groupby_pl():
    return (
        df_pl.filter(pl.col("COMP") == pl.lit(1))
        .group_by("DV_eletric")
        .agg(cs.by_name(vars_of_interest).mean())
    )


def filter_groupby_pd():
    return df_pd[df_pd["COMP"] == 1].groupby("DV_eletric").agg(vars_of_interest_pd)



def sort_pl():
    return df_pl.sort(by="timestamp")


def sort_pd():
    return df_pd.sort_values("timestamp")

In [10]:
# Prepare the benchmarking container
benchmark = pl.DataFrame()

# Running the tests
benchmark = measure_time(
    func=describe_pl,
    trials=20,
    container=benchmark,
    metadata={"package": "polars", "task": "describe"},
)
benchmark = measure_time(
    func=describe_pd,
    trials=20,
    container=benchmark,
    metadata={"package": "pandas", "task": "describe"},
)

benchmark = measure_time(
    func=read_csv_pl,
    trials=20,
    container=benchmark,
    metadata={"package": "polars", "task": "read_csv"},
)
benchmark = measure_time(
    func=read_csv_pd,
    trials=20,
    container=benchmark,
    metadata={"package": "pandas", "task": "read_csv"},
)

benchmark = measure_time(
    func=filter_groupby_pl,
    trials=20,
    container=benchmark,
    metadata={"package": "polars", "task": "filter_groupby"},
)

benchmark = measure_time(
    func=filter_groupby_pd,
    trials=20,
    container=benchmark,
    metadata={"package": "pandas", "task": "filter_groupby"},
)


benchmark = measure_time(
    func=sort_pl,
    trials=20,
    container=benchmark,
    metadata={"package": "polars", "task": "sort"},
)

benchmark = measure_time(
    func=sort_pd,
    trials=20,
    container=benchmark,
    metadata={"package": "pandas", "task": "sort"},
)

In [11]:

def plot_benchmark(data, title):

    # Group by package and task, calculate mean and standard error
    df_grouped = data.group_by(["package", "task"]).agg(
        mean_time=pl.mean("time"),
        std_error=pl.std("time"),
    ).with_columns(pl.col("package").replace({"polars": "Polars", "pandas": "Pandas"}))


    # Define the custom order
    custom_order = ['read_csv', 'describe', 'filter_groupby', 'sort']
    df_order = pl.DataFrame({
        'task': custom_order,
        'order': list(range(len(custom_order)))
    })
    df_grouped = df_grouped.join(df_order, on='task')
    df_grouped = df_grouped.sort('order')
    df_grouped = df_grouped.drop('order')

    # Create the bar plot

    fig = px.bar(
        df_grouped,
        x="task",
        y="mean_time",
        color="package",
        color_discrete_map={"Pandas": "#e4048c", "Polars": "#107cff"},
        barmode="group",
        #error_y="std_error",

    )

    fig.update_layout(
        {   "height": 630,
            "width": 1200,
            "plot_bgcolor": "rgba(0, 0, 0, 0)",
            "paper_bgcolor": "rgb(37, 50, 59)",  # dark blue background
            "legend": {
                "font": {"color": "white", "size": 25},
                "title_text": "",
                "yanchor": "top",
                "y": 0.8,
                "xanchor": "center",
                "x": 0.5,
            },  # white legend
            "title": {
                "text": title,
                "font": {"color": "white", "size": 40},  # white title
                "x": 0.5,  # center title
                "xanchor": "center",  # center title
            },
            "margin": {"l": 100, "r": 100, "t": 100, "b": 100},  # increase margins
        }
    )

    # Add text labels
    fig.for_each_trace(lambda trace: trace.update(text = [f'{y:.2f}s' for y in trace.y], textposition='outside', textfont=dict(color='white', size=20)))


    # Turn off grid lines and set axis labels and title to white
    fig.update_xaxes(title_text="", showgrid=False, tickfont=dict(size=20, color='white'))
    fig.update_yaxes(title_text="", showticklabels=False, showgrid=False, title_font=dict(size=25, color='white'), tickfont=dict(size=20, color='white'))

    # Increase the y-axis range by 10% to create space for the label of the highest bar
    max_y = df_grouped['mean_time'].max()
    fig.update_yaxes(range=[0, max_y * 1.1])

    # Remove lines around bars
    fig.update_traces(marker_line_width=0, marker_line_color='rgba(0, 0, 0, 0)')


    fig.show()

In [60]:
def summarize_benchmark(data):
    df_grouped = data.group_by(["package", "task"]).agg(
        mean_time=pl.mean("time"),
        std_error=pl.std("time"),
    ).with_columns(pl.col("package").replace({"polars": "Polars", "pandas": "Pandas"}))

    df_summary = (df_grouped
                    .pivot(index="task", columns=['package'], values='mean_time')
                    .rename({"Polars": "polars", "Pandas": "pandas"})
                    .with_columns([(pl.col('pandas') / pl.col('polars')).alias('pandas_to_polars_ratio'),
                                pl.col('pandas').sum().over(1).alias('pandas_total'),
                                pl.col('polars').sum().over(1).alias('polars_total'),
                                ])
                    .with_columns((pl.col('pandas_total') / pl.col('polars_total')).alias('pandas_to_polars_totals_ratio'))

    )


    print(df_summary)

In [61]:
compute = "Compute: Colab" if COLAB else "Compute: Macbook Pro 2018"
cores_no = multiprocessing.cpu_count()

plot_benchmark(data=benchmark, title=f"Benchmarking Wall Time <br><sup>{compute} with {cores_no} cores</sup>",)
summarize_benchmark(data=benchmark)

shape: (4, 7)
┌───────────────┬──────────┬──────────┬───────────────┬──────────────┬──────────────┬──────────────┐
│ task          ┆ polars   ┆ pandas   ┆ pandas_to_pol ┆ pandas_total ┆ polars_total ┆ pandas_to_po │
│ ---           ┆ ---      ┆ ---      ┆ ars_ratio     ┆ ---          ┆ ---          ┆ lars_totals_ │
│ str           ┆ f64      ┆ f64      ┆ ---           ┆ f64          ┆ f64          ┆ ratio        │
│               ┆          ┆          ┆ f64           ┆              ┆              ┆ ---          │
│               ┆          ┆          ┆               ┆              ┆              ┆ f64          │
╞═══════════════╪══════════╪══════════╪═══════════════╪══════════════╪══════════════╪══════════════╡
│ sort          ┆ 0.061225 ┆ 0.888086 ┆ 14.505244     ┆ 6.82918      ┆ 0.603365     ┆ 11.318495    │
│ read_csv      ┆ 0.302665 ┆ 5.039953 ┆ 16.651912     ┆ 6.82918      ┆ 0.603365     ┆ 11.318495    │
│ filter_groupb ┆ 0.06423  ┆ 0.201774 ┆ 3.141437      ┆ 6.82918      ┆ 0.6033

In [62]:
# Plotting the published benchmark results for Colab
published_colab_benchmark = pl.read_csv(
      "https://gist.github.com/gox6/7f1186585f79da7327b8b4cd796337f6/raw/benchmark-polars-vs-pandas-on-colab.csv"
  )

plot_benchmark(data=published_colab_benchmark, title=f"Benchmarking Wall Time <br><sup>Published Results for Colab with 2 cores</sup>",)
summarize_benchmark(data=published_colab_benchmark)

shape: (4, 7)
┌───────────────┬──────────┬──────────┬───────────────┬──────────────┬──────────────┬──────────────┐
│ task          ┆ polars   ┆ pandas   ┆ pandas_to_pol ┆ pandas_total ┆ polars_total ┆ pandas_to_po │
│ ---           ┆ ---      ┆ ---      ┆ ars_ratio     ┆ ---          ┆ ---          ┆ lars_totals_ │
│ str           ┆ f64      ┆ f64      ┆ ---           ┆ f64          ┆ f64          ┆ ratio        │
│               ┆          ┆          ┆ f64           ┆              ┆              ┆ ---          │
│               ┆          ┆          ┆               ┆              ┆              ┆ f64          │
╞═══════════════╪══════════╪══════════╪═══════════════╪══════════════╪══════════════╪══════════════╡
│ read_csv      ┆ 1.595724 ┆ 6.030984 ┆ 3.779466      ┆ 8.851415     ┆ 3.40629      ┆ 2.59855      │
│ describe      ┆ 1.054504 ┆ 1.14311  ┆ 1.084027      ┆ 8.851415     ┆ 3.40629      ┆ 2.59855      │
│ sort          ┆ 0.332449 ┆ 1.439027 ┆ 4.328561      ┆ 8.851415     ┆ 3.4062

In [58]:
# Plotting the published benchmark results for Macbook Pro 2018 with 12 cores

published_macbook_benchmark = pl.read_csv(
      "https://gist.github.com/gox6//c48d57d47c23c1dd1736286fbd78ec27/raw/benchmark-polars-vs-pandas-on-macbook.csv"
  )

plot_benchmark(data=published_macbook_benchmark, title=f"Benchmarking Wall Time <br><sup>Compute Macbook 2018 with 12 cores</sup>",)
summarize_benchmark(data=published_macbook_benchmark)

<html>
<body>
    <div style="display: flex;">
            <a href="https://medium.com/@jgrygolec" style="width: 45em; height: 15em; background-color: white; padding-left: 1em; display: flex; flex-direction: column; justify-content: center; color: inherit; text-decoration: none;">
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">Thank you for attention!</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849; font-weight: bold;">Click to see more of my articles on Medium.</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">BR,</p>
            <p style="margin: 0; font-size: 1.5em; color: #3D4849;">Jarek Grygolec</p>
        </a>
        <a href="https://medium.com/@jgrygolec" style="width: 30em; height: 15em; background-repeat: no-repeat; background-size: cover; background-position: center;">
                    <img src="https://miro.medium.com/v2/resize:fit:3992/0*WCQwsoZC0FA2-haq" style="width: 100%; height: 100%; object-fit: cover;"></a>
    </div>
</body>
</html>